##### Simulationen für MCAR mit den Top5 erklärenden Variablen (standardisiert)
In dem Notebook 'ols_prep' wird eine lineare Regression des Laptop-Preises auf alle Exogenen durchgeführt.
Anschließend werden alle Betas standardisiert und absteigend nach ihrer (absoluten) Größe sortiert. Daraus resultiert der Erklärungsgehalt der verschiedenen Exogenen. Wir nehmen die fünf Exogenen, die den größten Erklärungsgehalt haben und standardisieren sie. Sie dienen als Grundlage für die Simulationen in diesem Notebook. 
Es werden MCAR-Simulationen mit k=1000 durchgeführt, wobei jeweils nur die Top5 standardisierten Exogenen verwendet werden.
Es wird kein Jitter hinzugefügt.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf

rng = np.random.RandomState(0)

In [2]:
# Initialisierung von DataFrames und Data Cleaning
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index

# Nur die 5 Spalten auswählen, die auch den größten Erklärungsgehalt haben
selected_columns = ['Preis', 'PPI_Display', 'Marke_HP', 'Pixelshader', 'Betriebssystem_OHNE', 'Marke_Dell']

# Create a new DataFrame with only the selected columns
df = df[selected_columns]

df_noPrice = df.drop('Preis', axis=1)
imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])
imputed_stats_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'], index = np.arange(0.1, 1, 0.1))

imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])
imputed_stats_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'], index = np.arange(0.1, 1, 0.1))

no_imputation = pd.DataFrame(columns = ['SE', 'Alpha'])

# Skalierung (Standardisierung) von df_noPrice
col_names = df_noPrice.columns
scaler = StandardScaler().fit(df_noPrice.values)
df_noPrice = scaler.transform(df_noPrice.values)
df_noPrice = pd.DataFrame(df_noPrice, columns=col_names)

# Skalierung (Standardisierung) von df mit Preis (Preis ist unverändert)
df_std = df_noPrice.copy()
df_std.insert(0, 'Preis', df['Preis'])

# Wahrer Mittelwert:
true_mean = df['Preis'].mean()

In [3]:
def contains_mean(values):
    return 1 if (true_mean > (np.mean(values) - 1.95996 * stats.sem(values))) and (true_mean < (np.mean(values) + 1.95996 * stats.sem(values))) else 0

In [4]:
# OLS Model 
# fitting the model 
# df_noPrice ist bereits standardisiert
model = sm.OLS(df['Preis'], sm.add_constant(df_noPrice)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.532
Method:                 Least Squares   F-statistic:                     236.3
Date:                Fri, 22 Mar 2024   Prob (F-statistic):          3.43e-168
Time:                        11:31:53   Log-Likelihood:                -7945.9
No. Observations:                1038   AIC:                         1.590e+04
Df Residuals:                    1032   BIC:                         1.593e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1333.9633    

In [5]:
print(df.corr().to_latex(escape=False, float_format="%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  Preis &  PPI_Display &  Marke_HP &  Pixelshader &  Betriebssystem_OHNE &  Marke_Dell \\
\midrule
Preis               &   1.00 &         0.52 &     -0.11 &         0.46 &                -0.14 &        0.04 \\
PPI_Display         &   0.52 &         1.00 &     -0.11 &        -0.03 &                -0.07 &        0.13 \\
Marke_HP            &  -0.11 &        -0.11 &      1.00 &         0.04 &                 0.11 &       -0.30 \\
Pixelshader         &   0.46 &        -0.03 &      0.04 &         1.00 &                 0.08 &        0.05 \\
Betriebssystem_OHNE &  -0.14 &        -0.07 &      0.11 &         0.08 &                 1.00 &       -0.09 \\
Marke_Dell          &   0.04 &         0.13 &     -0.30 &         0.05 &                -0.09 &        1.00 \\
\bottomrule
\end{tabular}



C:\Users\Admin\AppData\Local\Temp\ipykernel_3932\4022997472.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.corr().to_latex(escape=False, float_format="%.2f"))


In [20]:
def del_ran(df_exog, labels, chance):
    rand_array = np.random.rand(df_exog.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    
    return [df_exog[delete_entries], labels[delete_entries], df_exog[keep_entries], labels[keep_entries]]

In [21]:
def impute_ols(test_values, test_labels, train_values, train_labels):
    
    # OLS Model
    # fitting the model 
    model = sm.OLS(train_labels, sm.add_constant(train_values)).fit() 

    imputed_values = model.predict(exog = sm.add_constant(test_values, has_constant='add')).tolist()
    return [np.mean(list(train_labels) + imputed_values), np.mean((imputed_values-test_labels)**2), stats.sem(list(train_labels) + imputed_values), contains_mean(list(train_labels) + imputed_values)]

In [22]:
def impute_knn(test_values, test_labels, train_values, train_labels):
    
    tree = KDTree(train_values.values, leaf_size=5)

    imputed_values_knn_1 = []
    imputed_values_knn_3 = []
    imputed_values_knn_5 = []

    for index, entry in enumerate(test_values.values):
 
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])

        imputed_values_knn_1.append(current_impute_knn_1)
        imputed_values_knn_3.append(current_impute_knn_3)
        imputed_values_knn_5.append(current_impute_knn_5)

    mean_knn_1 = np.mean(list(train_labels.values)+imputed_values_knn_1)
    mean_knn_3 = np.mean(list(train_labels.values)+imputed_values_knn_3)
    mean_knn_5 = np.mean(list(train_labels.values)+imputed_values_knn_5)

    mse_knn_1 = np.mean((test_labels.values - imputed_values_knn_1)**2)
    mse_knn_3 = np.mean((test_labels.values - imputed_values_knn_3)**2)
    mse_knn_5 = np.mean((test_labels.values - imputed_values_knn_5)**2)

    sem_knn_1 = stats.sem(list(train_labels.values)+imputed_values_knn_1)
    sem_knn_3 = stats.sem(list(train_labels.values)+imputed_values_knn_3)
    sem_knn_5 = stats.sem(list(train_labels.values)+imputed_values_knn_5)

    alpha_knn_1 = contains_mean(list(train_labels.values)+imputed_values_knn_1)
    alpha_knn_3 = contains_mean(list(train_labels.values)+imputed_values_knn_3)
    alpha_knn_5 = contains_mean(list(train_labels.values)+imputed_values_knn_5)


    return [mean_knn_1, mse_knn_1, sem_knn_1, alpha_knn_1, mean_knn_3, mse_knn_3, sem_knn_3, alpha_knn_3, mean_knn_5, mse_knn_5, sem_knn_5, alpha_knn_5]

In [23]:
def simul_knn():
    for c in np.arange(0.1, 1, 0.1):

        imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_knn.at[i] = impute_knn(temp[0], temp[1], temp[2], temp[3])

        return_values = []
        
        for column in imputed_simul_knn.columns:
            column_mean = np.mean(imputed_simul_knn[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_knn.loc[c] =  return_values
    imputed_stats_knn

simul_knn()
imputed_stats_knn

100%|██████████| 1000/1000 [01:50<00:00,  9.02it/s]


,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1327.722986,305194.10626,23.180244,1.0,1331.285095,215362.497696,23.041189,1.0,1330.063882,203790.12295,22.938032,1.0
0.2,1321.925217,311026.622398,23.143914,1.0,1328.910489,219546.608021,22.815244,1.0,1327.120904,207185.846072,22.613498,1.0
0.3,1316.191735,313567.410562,23.159729,0.922,1329.39906,223429.715687,22.567256,1.0,1326.629525,209420.15639,22.267547,1.0
0.4,1310.952574,321427.24122,23.187589,0.811,1331.776833,226108.766867,22.230882,0.985,1327.856451,213109.201084,21.856763,0.998
0.5,1312.01055,325459.594133,23.222975,0.709,1334.84581,230396.603803,21.868279,0.953,1330.728369,218402.016729,21.391132,0.978
0.6,1319.034149,329180.475217,23.168113,0.709,1340.428312,237718.967331,21.416496,0.87,1335.511722,226254.269079,20.835883,0.933
0.7,1326.434172,343338.325878,23.134602,0.657,1344.99063,248468.838794,20.914742,0.781,1339.542007,237430.491172,20.243141,0.841
0.8,1337.354397,358815.775073,22.918459,0.563,1346.008948,262770.257738,20.28026,0.685,1341.150183,252127.754042,19.408015,0.731
0.9,1346.723035,398136.633994,22.603062,0.432,1333.541097,296208.624415,19.078286,0.503,1324.522829,291455.344438,17.347879,0.45


In [25]:
def simul_ols():
    
    for c in np.arange(0.1, 1, 0.1):
        
        imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_ols.at[i] = impute_ols(temp[0], temp[1], temp[2], temp[3])

        return_values = []

        for column in imputed_simul_ols.columns:
            column_mean = np.mean(imputed_simul_ols[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_ols.loc[c] =  return_values
    imputed_stats_ols

simul_ols()

 19%|█▉        | 193/1000 [00:01<00:05, 155.43it/s]


ValueError: shapes (929,6) and (5,) not aligned: 6 (dim 1) != 5 (dim 0)

In [26]:
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1).to_csv('MCAR_Simulation_Top5-10')
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1)

,Mean,MSE OLS,SE OLS,Alpha OLS,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1333.766074,264709.866489,22.689205,1.0,1327.722986,305194.10626,23.180244,1.0,1331.285095,215362.497696,23.041189,1.0,1330.063882,203790.12295,22.938032,1.0
0.2,1333.833626,264502.743889,22.121756,1.0,1321.925217,311026.622398,23.143914,1.0,1328.910489,219546.608021,22.815244,1.0,1327.120904,207185.846072,22.613498,1.0
0.3,1334.032109,263997.453895,21.557788,1.0,1316.191735,313567.410562,23.159729,0.922,1329.39906,223429.715687,22.567256,1.0,1326.629525,209420.15639,22.267547,1.0
0.4,1334.060414,264999.402438,20.957072,0.997,1310.952574,321427.24122,23.187589,0.811,1331.776833,226108.766867,22.230882,0.985,1327.856451,213109.201084,21.856763,0.998
0.5,1334.562957,265833.715514,20.387155,0.991,1312.01055,325459.594133,23.222975,0.709,1334.84581,230396.603803,21.868279,0.953,1330.728369,218402.016729,21.391132,0.978
0.6,1334.645452,265906.736928,19.785527,0.958,1319.034149,329180.475217,23.168113,0.709,1340.428312,237718.967331,21.416496,0.87,1335.511722,226254.269079,20.835883,0.933
0.7,1334.066736,267402.124532,19.133415,0.873,1326.434172,343338.325878,23.134602,0.657,1344.99063,248468.838794,20.914742,0.781,1339.542007,237430.491172,20.243141,0.841
0.8,1333.846253,269939.4993,18.569379,0.738,1337.354397,358815.775073,22.918459,0.563,1346.008948,262770.257738,20.28026,0.685,1341.150183,252127.754042,19.408015,0.731
0.9,NaN,NaN,NaN,NaN,1346.723035,398136.633994,22.603062,0.432,1333.541097,296208.624415,19.078286,0.503,1324.522829,291455.344438,17.347879,0.45
